# SEIRD fixed infectious rate

In [ ]:
import os
import datetime

import pandas as pd
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='paper', style='darkgrid', font_scale=1.2)

In [ ]:
# all data
DATA_DIR = 'data'
FILE_NAME = 'initial_timeseries_SEIRD.csv' 
FILE_PATH = os.path.join(os.pardir, DATA_DIR, FILE_NAME)
df = pd.read_csv(FILE_PATH)

# derived data
cum_positives = np.cumsum(df.confirmed_positives.values)
cum_deceased = np.cumsum(df.confirmed_deceased.values)

# training_data
I_train = cum_positives - df.estimate_recovered.values
R_train = df.estimate_recovered.values
D_train = cum_deceased

# initial conditions
S0 = np.max(cum_positives) # missing something here ?
E0 = 10
I0 = 1
R0 = 0
D0 = 0
ic = [S0, E0, I0, R0, D0]

In [ ]:
# additional input params
beta = 0.5
pcr_sens = 0.9
pcr_spec = 0.95

In [ ]:
# TBA - fixed beta SEIRD model, use the coropy API look and feel

In [ ]:
model = SEIRDModel(loss_fn='mse')
params, loss = model.fit(I_train, R_train, D_train, ic)
(S_sim, E_sim, I_sim, R_sim, D_sim) = model.simulate()
fig = plt.figure(figsize=(9, 5))
ax = fig.subplots(nrows=1, ncols=1)
ax.plot(I_train, '.', c='b', label='$I_{train}$')
ax.plot(I_sim, '-', c='b', label='$I_{fit}$')
ax.plot(R_train, '.', c='r', label='$R_{train}$')
ax.plot(R_sim, '-', c='r', label='$R_{fit}$')
ax.plot(D_train, '.', c='g', label='$D_{train}$')
ax.plot(D_sim, '-', c='g', label='$D_{fit}$')
ax.text(0.18, 0.96,
        f'β = {params[0]:.6f}\nγ = {params[1]:.6f}\nδ = {params[2]:.6f}\nμ = {params[3]:.6f}', 
        transform=ax.transAxes,
        verticalalignment='top', 
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=1))
ax.set_xlabel('$\Delta t$')
ax.set_ylabel('$N$')
ax.legend()
plt.show();

In [ ]:
# beta <= average number of contacts per person per time 
# gamma <= infectious rate = 1/(infectious time period)
# R0 <= basic reproduction number
R0 = params[0] / params[1]
R0